# IMDB Investigation

### Import Library

In [1]:
# import
import numpy as np
import pandas as pd


### Import Data

In [2]:
basics_url='https://datasets.imdbws.com/title.basics.tsv.gz'
akas_url='https://datasets.imdbws.com/title.akas.tsv.gz'
ratings_url='https://datasets.imdbws.com/title.ratings.tsv.gz'

basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

### Review Dataframes

In [3]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
#creating backup of 'basics' dataframe
df_b_save = basics.copy()

In [5]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [6]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1976
1,tt0000002,5.8,264
2,tt0000003,6.5,1825
3,tt0000004,5.6,178
4,tt0000005,6.2,2617


### Preprocessing Cleanup

#### Title Basics

##### Find NAN Values

In [7]:
basics = basics.replace({'\\N':np.nan})

In [8]:
### Evaluating NAN values in columns

In [9]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1334561
endYear           9783550
runtimeMinutes    6969683
genres             444698
dtype: int64

In [10]:
#Display the percentage of missing values by row. 
print(basics.isna().sum()/len(basics)*100)

tconst             0.000000
titleType          0.000000
primaryTitle       0.000111
originalTitle      0.000111
isAdult            0.000010
startYear         13.493008
endYear           98.916059
runtimeMinutes    70.466607
genres             4.496095
dtype: float64


In [11]:
#Modifying end dates that are missing to arbitrary number
end_nan = 9999
basics['endYear'].fillna(value = end_nan, inplace = True)

##### Eliminate movies that are null for runtimeMinutes

In [12]:
basics.dropna(subset=['runtimeMinutes'], inplace=True)

##### Eliminate movies that are null for genre

In [13]:
basics.dropna(subset=['genres'], inplace=True)

##### keep only titleType==movie

In [14]:
basics = basics[basics['titleType'] == 'movie']

##### Eliminating NAN values for 'startYear'

In [15]:
basics.dropna(subset=['startYear'], inplace=True)

##### NAN values resolved

In [16]:
print(basics.isna().sum()/len(basics)*100)

tconst            0.0
titleType         0.0
primaryTitle      0.0
originalTitle     0.0
isAdult           0.0
startYear         0.0
endYear           0.0
runtimeMinutes    0.0
genres            0.0
dtype: float64


##### Modifying Datatypes to conform to data dictionary

In [17]:
basics = basics.astype({"tconst":"string","titleType":"string","primaryTitle":"string","originalTitle":"string", "isAdult":"int", "startYear":"int", "endYear": "int","runtimeMinutes":"int", "genres":"string"})

In [18]:
basics.dtypes

tconst            string
titleType         string
primaryTitle      string
originalTitle     string
isAdult            int32
startYear          int32
endYear            int32
runtimeMinutes     int32
genres            string
dtype: object

##### keep startYear 2000-2022

In [21]:
basics = basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)]

In [ ]:
##### Eliminate movies that include "Documentary" in genre